In [64]:
import numpy as np
import scipy as sp
import igl
import trimesh
import polyscope as ps
import os

from cotanLaplace import *
from massmatrix import *
from Mollification import *

root_folder = os.getcwd()

### TODO? Make the demos interactive (e.g. select source vertices for heat geodisics)

In [65]:
[V,F]= igl.read_triangle_mesh(os.path.join(
             root_folder, "../data/bad_triangle/", "holly_wand_complete_letter_A_Print.stl"))

#[V,F]= igl.read_triangle_mesh(os.path.join(
#            root_folder, "../data", "spot.obj"))

In [66]:
#init poly scope and show mesh
ps.init()
ps.set_up_dir('z_up')
ps.set_ground_plane_mode('tile_reflection')
#register mesh
#mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0], edge_width=0.9, edge_color=[0.2, 0.1, 0.8, 1.0])
#ps.show()

# Solve Poisson equation (interpolation)

In [4]:
# use cotan Laplace and Mass Matrix to solve Lx = -M(rho - rho0)

L = igl.cotmatrix(V, F)
M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)

# choose 2 random vertex indices
v1 = np.random.randint(0, V.shape[0])
v2 = np.random.randint(0, V.shape[0])

# rho is a column vector of length V.shape[0] and with all zeros except for rho[v1] and rho[v2]
rho = sp.zeros((1, V.shape[0]))
rho[:,v1] = 100
rho[:,v2] = -100

def solvePoisson(L, M, rho):
        totArea = sp.sum(M.diagonal())

        # rho0 is a column vector of length V.shape[0], with all the same value = sum(rho[i] * M[i,i]) / totArea
        rho0 = sp.sum(rho * M.diagonal()) / totArea # this is more general than just summing at the 2 vertices

        f = sp.sparse.linalg.spsolve(L, -M.dot(rho.T - rho0.T))

        return f

f = solvePoisson(L, M, rho)

# plot the result
mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0])
mesh.add_scalar_quantity("f", 
        f, defined_on='vertices', cmap="viridis")

ps.show()




C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_18648/1069685933.py:11: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead
  rho = sp.zeros((1, V.shape[0]))
C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_18648/1069685933.py:16: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  totArea = sp.sum(M.diagonal())
C:\Users\Hossam Saeed\AppData\Local\Temp/ipykernel_18648/1069685933.py:19: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
  rho0 = sp.sum(rho * M.diagonal()) / totArea # this is more general than just summing at the 2 vertices


## Mesh Smoothing

In [67]:
# use cotan Laplace matrix for smoothing
# solving (M - h * L) Vnew = M * Vold

mesh = ps.register_surface_mesh("mesh", V, F, color=[0.6, 0.4, 1.0, 1.0])
avg_edge_length = igl.avg_edge_length(V, F)
delta = 10 * avg_edge_length
E, eps, newL = IntrinsicMollification(V, F, delta)
L = cotanLaplace(F, newL, neg_hack=NEG_HACK.NONE, nan_hack=NAN_HACK.NONE)

#L = igl.cotmatrix(V, F)
h = 0.2 * avg_edge_length

t = 0
n = 2

def smooth():
    global t
    global n
    global V
    global mesh

    if t < n:
        newL = igl.edge_lengths(V, F)
        M = massmatrix(newL, F, MASSMATRIX_TYPE.BARYCENTRIC)
        #M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)
        #M = igl.massmatrix_intrinsic(newL, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)
        A = M - h * L

        for i in range(3):
            V[:,i] = sp.sparse.linalg.spsolve(A, M.dot(V[:,i])) # Todo look for a way to use a positive semi-definite solver

        mesh.update_vertex_positions(V)
        t += 1
        print("Smoothing iteration: ", t)

ps.set_user_callback(smooth)

ps.show()

ps.clear_user_callback()



Smoothing iteration:  1
Smoothing iteration:  2


## Next: Heat Method for Geodesic Distances

In [27]:
# L = igl.cotmatrix(V, F)
# M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)
# h = 0.005
# A = M + h * L

# print(sp.sparse.linalg.norm(M))
# I = sp.sparse.identity(M.shape[0])
# print(I.dot(V[:, 0]) - V[:, 0])
# print(sp.sparse.linalg.norm(sp.sparse.identity(M.shape[0])))


0.08031036303393863
[0. 0. 0. ... 0. 0. 0.]
119.54078801814885


In [12]:
L = igl.edge_lengths(V, F)         # columns correspond to edges lengths [1,2],[2,0],[0,1]

M_ours_i = massmatrix(L, F, MASSMATRIX_TYPE.BARYCENTRIC).todense()
M_igl = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC).todense()
M_igl_i = igl.massmatrix_intrinsic(L, F, igl.MASSMATRIX_TYPE_BARYCENTRIC).todense()
print(np.linalg.norm(M_ours_i - M_igl_i))
print(np.linalg.norm(M_igl - M_igl_i))


6.928722052989442e-18
0.0
